In [3]:
import pandas as pd
import json
from pydantic import BaseModel, Field, ValidationError, field_validator
from typing import List, Dict, Any
import ast

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.llms import Ollama

In [4]:
listings = pd.read_csv('data/data_uncategorised_Oct21_to_Jan22.csv')
#listings.head()
unique_listings=pd.DataFrame(listings['drug_title'].unique(),columns=['listing'])

/var/folders/_0/s_2t6l510c3bdxngny7zmfyc0000gn/T/ipykernel_6939/247093999.py:1: DtypeWarning: Columns (8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  listings = pd.read_csv('data/data_uncategorised_Oct21_to_Jan22.csv')


In [5]:
def dataframe_batches(df, batch_size=10):
    column_name = df.columns[0]
    total_rows = len(df)
    results = []

    for start in range(0, total_rows, batch_size):
        end = min(start + batch_size, total_rows)
        batch = df.iloc[start:end]
        
        batch_result = {str(element): [] for element in batch[column_name]}
        results.append(batch_result)

    return results

class ResponseChecks(BaseModel):
    data: Dict[str, List[Any]]
    
    @field_validator("data")
    def check_dict_format(cls, value):
        assert isinstance(value, dict), "Response must be a dictionary"
        assert len(value) > 0, "Dictionary must not be empty"
        for key, val in value.items():
            assert isinstance(val, list), f"Value for key '{key}' must be a list"
        return value

def execute_keyword_LLM(list_of_dicts, chain):
    results = []
    error_listings = []
    total_items = len(list_of_dicts)
    
    for index, item in enumerate(list_of_dicts, 1):
        # Display progress every 10 runs
        if index % 10 == 0 or index == total_items:
            print(f"Progress: {index}/{total_items} items processed")
        
        try:
            response = chain.run(test_data=item)
            output = ast.literal_eval(response)
            
            # Validate the output
            ResponseChecks(data=output)
            
            # If validation passes, add to results
            for key, value in output.items():
                results.append({"key": key, "value": value})
        
        except Exception as e:
            print(f"Error processing item {index}: {str(e)}")
            error_listings.append(item)
            continue
    
    # Create DataFrame from results
    df = pd.DataFrame(results)
    
    # Print summary
    print(f"\nProcessing complete.")
    print(f"Total items: {total_items}")
    print(f"Successful: {len(list_of_dicts) - len(error_listings)}")
    print(f"Errors: {len(error_listings)}")
    
    return df, error_listings

In [6]:
#Break the data frame into batches and convert it to dict for processing
list_of_dicts=dataframe_batches(unique_listings)

In [7]:
list_of_dicts = list_of_dicts[:500]

In [8]:
# Define the input and output formats
input_format = r"""{{
  'listing text here': [],
  'listing text here': [],
  ...
}}"""

output_format = r"""{{
  'listing text here': ['keyword1', 'keyword2', ...],
  'listing text here': ['keyword1', 'keyword2', ...],
  ...
}}"""

example_input = r"""{{
  '1 Blister Codein 50mg Codeinium': [],
  '65 sample - choose what u want, mix and match': [],
  'SPECIAL PROMO: 500x S903-4mg Hulk Bars': [],
  '7G L.S.D A+++ TOP PRICES IN UK!!! NDD-FREE POSTAGE': [],
  '3.5-56G GREEN CRACKA+++ TOP PRICES IN UK!!! NDD-FREE POSTAGE': [],
  'Pure Cocaine 98% purity, 5 grams available': [],
  'Acid tabs, blotters, LSD, 25mg each': [],
  '50 grams Cannabis buds - premium quality': [],
  'Xanax bars, 10mg - pharma grade': [],
  'Variety of pills and powders, 100% satisfaction guaranteed': [],
  'BD N BSM TEST BUNDLE 1OZ -indoor only- A++': []
}}"""

example_output = r"""{{
  '1 Blister Codein 50mg Codeinium': ['Codein', 'Codeinium'],
  '65 sample - choose what u want, mix and match': ['NA'],
  'SPECIAL PROMO: 500x S903-4mg Hulk Bars': ['S903', 'Hulk Bars'],
  '7G L.S.D A+++ TOP PRICES IN UK!!! NDD-FREE POSTAGE': ['LSD'],
  '3.5-56G GREEN CRACKA+++ TOP PRICES IN UK!!! NDD-FREE POSTAGE': ['green crack'],
  'Pure Cocaine 98% purity, 5 grams available': ['Cocaine'],
  'Acid tabs, blotters, LSD, 25mg each': ['Acid', 'LSD'],
  '50 grams Cannabis buds - premium quality': ['Cannabis'],
  'Xanax bars, 10mg - pharma grade': ['Xanax'],
  'Variety of pills and powders, 100% satisfaction guaranteed': ['NA'],
  'BD N BSM TEST BUNDLE 1OZ -indoor only- A++': ['BD', 'BSM']
}}"""

# Define the template for the prompt
template = f"""Act as an NLP specialist, and your task is to extract keywords from the given substance listings scraped from the dark web. Keywords can include the main item of the listing, and/or any chemical name or compound mentioned, and/or any slang term related to illicit substances. General or common words in the listings like ‘ounces,’ ‘variety,’ ‘after,’ etc., are not considered keywords unless they are part of a recognized slang term. There can be more than 1 keyword in a given listing.

Input format:
{input_format}

Output format:
{output_format}

Example

Input:
{example_input}

Output:
{example_output}

Now perform the task on the following and strictly follow the output format and task instructions given above. Remember there can be more than one keywords as well for a given listing. Please don't hallucinate and don't extract imaginary keywords. Listings data:

{{test_data}}
"""

In [9]:
# Create a PromptTemplate instance
prompt = PromptTemplate(template=template, input_variables=["test_data"])

# Initialize the Ollama LLM with the correct model name
llm = Ollama(model="mistral-nemo")

# Create the LLMChain
chain = LLMChain(llm=llm, prompt=prompt)

# Run the chain with the first element of list_of_dicts
#response = chain.run(test_data=list_of_dicts[0])
#try:
#    output=ast.literal_eval(response)
#except:
#    output=''
#    print('Invalid Response')
#print(response)

/Users/vanditsadaphale/Library/Caches/pypoetry/virtualenvs/llama-fine-tuning-rZexxVwI-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [10]:
result_df, errors_df = execute_keyword_LLM(list_of_dicts, chain)


/Users/vanditsadaphale/Library/Caches/pypoetry/virtualenvs/llama-fine-tuning-rZexxVwI-py3.9/lib/python3.9/site-packages/langchain_core/_api/deprecation.py:141: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Progress: 10/500 items processed
Progress: 20/500 items processed
Progress: 30/500 items processed
Progress: 40/500 items processed
Progress: 50/500 items processed
Progress: 60/500 items processed
Progress: 70/500 items processed
Progress: 80/500 items processed
Progress: 90/500 items processed
Progress: 100/500 items processed
Progress: 110/500 items processed
Progress: 120/500 items processed
Progress: 130/500 items processed
Progress: 140/500 items processed
Progress: 150/500 items processed
Progress: 160/500 items processed
Error processing item 166: 1 validation error for ResponseChecks
data
  Input should be a valid dictionary [type=dict_type, input_value={"'50G Sgt-78 #5F-MDMB #C...: ['Replica', 'Carts']"}, input_type=set]
    For further information visit https://errors.pydantic.dev/2.8/v/dict_type
Progress: 170/500 items processed
Progress: 180/500 items processed
Progress: 190/500 items processed
Error processing item 193: EOL while scanning string literal (<unknown>, line 2

In [11]:
#result_df.tail(60)

In [12]:
import os

output_folder = 'output'
file_name = 'mistral-nemo_10.csv'
file_path = os.path.join(output_folder, file_name)

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if os.path.exists(file_path):
    result_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    result_df.to_csv(file_path, index=False)

In [13]:
list_of_dicts=dataframe_batches(unique_listings)
list_of_dicts = list_of_dicts[501:1300]
result_df, errors_df = execute_keyword_LLM(list_of_dicts, chain)

import os

output_folder = 'output'
file_name = 'mistral-nemo_10.csv'
file_path = os.path.join(output_folder, file_name)

# Create the output folder if it doesn't exist
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

if os.path.exists(file_path):
    result_df.to_csv(file_path, mode='a', header=False, index=False)
else:
    result_df.to_csv(file_path, index=False)

Progress: 10/799 items processed
Progress: 20/799 items processed
Progress: 30/799 items processed
Progress: 40/799 items processed
Progress: 50/799 items processed
Progress: 60/799 items processed
Progress: 70/799 items processed
Progress: 80/799 items processed
Progress: 90/799 items processed
Progress: 100/799 items processed
Progress: 110/799 items processed
Progress: 120/799 items processed
Progress: 130/799 items processed
Progress: 140/799 items processed
Progress: 150/799 items processed
Progress: 160/799 items processed
Progress: 170/799 items processed
Progress: 180/799 items processed
Progress: 190/799 items processed
Progress: 200/799 items processed
Progress: 210/799 items processed
Progress: 220/799 items processed
Progress: 230/799 items processed
Progress: 240/799 items processed
Progress: 250/799 items processed
Progress: 260/799 items processed
Progress: 270/799 items processed
Progress: 280/799 items processed
Progress: 290/799 items processed
Progress: 300/799 items